In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
from jax.random import PRNGKey
from hyperiax.execution import UnorderedExecutor
from hyperiax.tree.fasttree import FastTree
from hyperiax.models import UpdateLambda
import jax
from hyperiax.tree.builders import symmetric_topology
from jax import numpy as jnp


In [26]:
key = PRNGKey(1337)
depth = 3
print("Number of nodes:", (2<<depth)-1)

Number of nodes: 15


In [35]:
topology = symmetric_topology(depth, 2)
tree = FastTree(topology)
#tree = FastBiTree(depth)
tree.add_property('edge_length', (1,))
tree.add_property('value', (2,))

In [37]:
tree.data['edge_length'] = jnp.ones_like(tree.data['edge_length'])
tree.data['value'] = jax.random.normal(key, tree.data['value'].shape)

In [38]:
def update(child_value, parent_value, value, **kwargs):
    return {'value': (child_value+parent_value+value)/3}

def up(value, **kwargs):
    return {'value': value}

model = UpdateLambda(up, update, reductions={'value': 'sum'})

In [39]:
exe = UnorderedExecutor(model)

In [134]:
exe.update(tree)

{'edge_length': Array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=float32),
 'value': Array([[-2.0376794e+06, -6.3445588e+05],
        [-1.5216094e+06, -4.7377031e+05],
        [-1.5216092e+06, -4.7377225e+05],
        [-6.1023669e+05, -1.9000417e+05],
        [-6.1023844e+05, -1.9000338e+05],
        [-6.1023775e+05, -1.9000584e+05],
        [-6.1023719e+05, -1.9000561e+05],
        [ 1.3846554e+00, -9.2566572e-02],
        [-6.5286350e-01, -5.9667703e-02],
        [-1.6942079e+00, -2.0053019e-01],
        [-9.7452837e-01,  1.6766815e+00],
        [-7.1187401e-01, -1.0034498e+00],
        [-7.9270500e-01, -5.2675545e-01],
        [-3.9856440e-01,  4.1108310e-01],
        [ 9.4253209e-04, -1.4827111e+00]], dtype=float32)}